## Load packages

In [107]:
# !pip install efficientnet
# !pip install pydot
# !pip install graphviz
# !pip install pydotplus
import tqdm
from tqdm import tqdm_notebook
import math
from PIL import Image
# import pydot
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow.keras.backend as K
# import pydotplus
# from pydotplus import graphviz
# import pydot
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.utils.data_utils import Sequence
# import efficientnet.tfkeras as efn
from keras.applications import VGG16
tf.keras.backend.clear_session()
# BytesList = tf.train.BytesList
# FloatList = tf.train.FloatList
# Int64List = tf.train.Int64List
# Feature = tf.train.Feature
# Features = tf.train.Features
# Example = tf.train.Example

# to make this notebook's output stable across runs
# np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


print(f"Tensor flow version --> {tf.__version__}")
# from tensorflow.keras.utils import Sequence
print(f"Keras version --> {keras.__version__}")

# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


Tensor flow version --> 2.3.0
Keras version --> 2.4.0


## Input parameters

In [108]:
# path parameters
OUTPUT_PATH = "../input/osiccsvfiles/osic_csv_files/"
OUTPUT_PATH_LUNG_MASK_IMAGES_1MM = "../input/osic-ads-images/lung_mask_1mm/"

# augmentation parameters
ROT_ = 0
SHR_ = 2
HZOOM_ = 4
WZOOM_ = 4
HSHIFT_ = 4
WSHIFT_ = 4


# ROT_ = 0
# SHR_ = 2.0
# HZOOM_ = 8.0
# WZOOM_ = 8.0
# HSHIFT_ = 8.0
# WSHIFT_ = 8.0

# input image size
image_size = 299

# batch size
bs = 16

# FOLDS PARAMETERS
fold = 0 # which fold to run
n_folds = 10 # CSV FILE

# FINE TUNING PARAMETERS
# Number of layers to fine-tune of base model
trainable = 8
early_stopping_fine_tuning = 8

# MODEL PARAMATERS AND LEARNING RATE
EPOCHS = 100
optimizer = keras.optimizers.Adam(lr=0.00125, beta_1=0.9, beta_2=0.999)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)
checkpoint_cb = keras.callbacks.ModelCheckpoint(f"/kaggle/working/osic_pat_level_image_index_{n_folds}FOLDS_{EPOCHS}E_f_{fold}.h5",
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# AUGMENTATION PARAMETER
TTA_STEPS = 11

# Test Patients
patients_test = ["ID00419637202311204720264", "ID00421637202311550012437", "ID00422637202311677017371", "ID00423637202312137826377", "ID00426637202313170790466"]

# Junk Patients
junk_patients = [ "ID00170637202238079193844","ID00090637202204766623410"] 


########################## PLAYGROUND ########################################
# AUG_BATCH = 16
# patients_image_perc_20 = ["ID00168637202237852027833","ID00099637202206203080121", 
#                           "ID00285637202278913507108", "ID00371637202296828615743", 
#                           "ID00186637202242472088675"]

# patients_image_perc_70 = ["ID00123637202217151272140", "ID00108637202209619669361",
#                           "ID00190637202244450116191", "ID00135637202224630271439",
#                           "ID00020637202178344345685", "ID00027637202179689871102",
#                           "ID00035637202182204917484", "ID00109637202210454292264",
#                           "ID00233637202260580149633", "ID00089637202204675567570",
#                           "ID00032637202181710233084", "ID00393637202302431697467",
#                           "ID00075637202198610425520", "ID00067637202189903532242"]

# no good percentile index [0.2, 0.3, 0.7] for these 2 Patient 
#"ID00094637202205333947361", #"ID00086637202203494931510",
#"ID00067637202189903532242", "ID00014637202177757139317", \ # "ID00240637202264138860065", #  "ID00122637202216437668965"

# print(f"# of Patients with 20th Percentile Image as Input to pre-trained model --> {len(set(patients_image_perc_20))}")
# print(f"# of Patients with 70th Percentile Image as Input to pre-trained model --> {len(set(patients_image_perc_70))}")



## UDFs

In [109]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))


def transform(image, image_size):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = image_size[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    
    idx = tf.stack([x,y,z])    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)

    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])


def score(y_true, y_pred):
    C1 = tf.constant(70, dtype='float32')
    C2 = tf.constant(1000, dtype="float32")
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return keras.backend.mean(metric)

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return keras.backend.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss

def convert_to_tensor(arg):
    out = tf.convert_to_tensor(arg, dtype=tf.float32)
    return out

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")    

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def cutmix_mixup(image):
    DIM = IMAGE_SIZE[0]
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2 = cutmix(image, CUTMIX_PROB)
    image3 = mixup(image,MIXUP_PROB)
    imgs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    return image4

def cutmix(image, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    
    imgs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    return image2

def mixup(image, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    
    imgs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    return image2


# def preprocess(image):

# #     if aug:
# #         image = tf.image.random_flip_left_right(image)
#             #img = tf.image.random_hue(img, 0.01)
# #         image = tf.image.random_saturation(image, 0.7, 1.3)
# #         image = tf.image.random_contrast(image, 0.8, 1.2)
# #         image = tf.image.random_brightness(image, 0.1)  
#     resized_image = tf.image.resize(image, [image_size, image_size])
#     final_image = keras.applications.xception.preprocess_input(resized_image*255)
    
#     return final_image

class DataGenerator(Sequence):
    
    def __init__(self, dataset, batch_size, aug, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.aug = aug
        self.dataset = dataset
        self.shuffle = shuffle
        self.indexes = np.arange(self.dataset.shape[0])
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.dataset.shape[0]/self.batch_size))
 
 
    def __getitem__(self, index):
        # Generate indexes of the batch
        idxs = [i for i in range(index*self.batch_size,(index+1)*self.batch_size)]
 
        # Find list of IDs
        ids_tmp = [self.indexes[k] for k in idxs]
        
        # Generate data
        # tabular
        tab = self.dataset.loc[ids_tmp, ['Male', 'Female', 'Ex-smoker', 'Never smoked', 
                                         'Currently smokes', 'age', 'week', 'base_fvc']].to_numpy(dtype=np.float32)
        
        # image
        patient_ids = list(self.dataset.loc[ids_tmp, 'Patient'])
        imgs_master = []
        
        for pat_id in patient_ids:    
            images = os.listdir(os.path.join(OUTPUT_PATH_LUNG_MASK_IMAGES_1MM, pat_id))
            sorted_images = sorted(images,key=lambda x: int(os.path.splitext(x)[0]))
#             slices = [Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L') for index, file in enumerate(sorted_images)]    
            
            
            ######## OLD ########
#             img1 = np.mean([np.array(slices[i]) for i in range(int(0.15 * len(slices)), int(0.4 * len(slices)))], axis=0)/255
#             img1 = tf.convert_to_tensor(img1.reshape((*img1.shape, 1)))
#             img2 = np.mean([np.array(slices[i]) for i in range(int(0.4 * len(slices)/2), int(0.65 * len(slices)))], axis=0)/255
#             img2 = tf.convert_to_tensor(img2.reshape((*img2.shape, 1)))
#             img3 = np.mean([np.array(slices[i]) for i in range(int(0.65 * len(slices)), int(0.9 * len(slices)))], axis=0)/255
#             img3 = tf.convert_to_tensor(img3.reshape((*img3.shape, 1)))

#             randint = np.random.randint(0,3)
#             if randint == 0:
#                 img_to_resize = img1
#             elif randint == 1:
#                 img_to_resize = img2
#             else:
#                 img_to_resize = img3

            ######## NEW Augementation ########
            ###### All processing on GPU!!! #######
        
            ##### FOR TEST PATIENTS ONLY ######
#             if pat_id in patients_test and self.aug==True:
#                 combo = [0.2, 0.3, 0.5, 0.7, 0.8]
#                 perc = np.random.choice(combo)
#                 print(perc)
#             else:
#                 combo = [0.2, 0.3, 0.5, 0.7, 0.8]
#                 perc = np.random.choice(combo)
              
############## NEW EXPERIMENT TO BE TRIED ###################             
#             combo = [0.2, 0.3, 0.7]
#             perc = np.random.choice(combo)
                

#             if pat_id in patients_image_perc_20:
#                 perc = 0.2
#             elif pat_id in patients_image_perc_70:
#                 perc = 0.7
#             elif pat_id in "ID00423637202312137826377":
#                 print("yes")
#                 combo = [0.4, 0.3]
#                 perc = np.random.choice(combo)
#                 print(perc)
#                 perc = 0.4
#             elif pat_id in "ID00426637202313170790466":
#                 print("yes")
#                 perc = 0.7
#             elif pat_id in "ID00421637202311550012437":
#                 perc = 0.2
            
    
#             #################### MONTAGE ####################
            combo = [0.2, 0.3, 0.5, 0.7, 0.8]
            perc = np.random.choice(combo, 4, replace=False)
            inputImages = []
            
            for perc_tmp in perc:
                index = int(len(sorted_images)*perc_tmp)
                image = tf.keras.preprocessing.image.img_to_array(Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L'))/25
            #     print(image.shape)
            #     image = image[:, :, 0]
                resized_image = tf.image.resize(image, [128, 128])
                inputImages.append(resized_image[:, :, 0])

            final_image = np.zeros((256, 256))
            final_image[0:128, 0:128] = inputImages[0]
            final_image[0:128, 128:256] = inputImages[1]
            final_image[128:256, 0:128] = inputImages[2]
            final_image[128:256, 128:256] = inputImages[3]
            image = final_image
            ############################################################
            
            # SAME AS OLD PART
            # print(image.shape)
            img_to_resize = tf.convert_to_tensor(image.reshape((*image.shape, 1)))
            # print(img_to_resize.shape)
            images = tf.convert_to_tensor([img_to_resize]*3)
            # print(images.shape)
            images_reduced_mean = tf.reduce_mean(images, axis=3)
            image_pre_proc = tf.stack([images_reduced_mean[0], images_reduced_mean[1], images_reduced_mean[2]], axis=-1)
            
            
            ############# old ####################
#              ##### FOR TEST PATIENTS ONLY ######
#             if pat_id in patients_test and self.aug==True:
#                 combo = [0.2, 0.3, 0.5, 0.7, 0.8]
#                 perc = np.random.choice(combo)
#                 print(perc)
#             else:
#                 combo = [0.2, 0.3, 0.5, 0.7, 0.8]
#                 perc = np.random.choice(combo)

#             index = int(len(sorted_images)*perc)
#             image = tf.keras.preprocessing.image.img_to_array(Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L'))/255
#             image = image[:, :, 0] 
#             img_to_resize = tf.convert_to_tensor(image.reshape((*image.shape, 1)))
#             images = tf.convert_to_tensor([img_to_resize]*3)
#             images_reduced_mean = tf.reduce_mean(images, axis=3)
#             image_pre_proc = tf.stack([images_reduced_mean[0], images_reduced_mean[1], images_reduced_mean[2]], axis=-1)

            # With Augmentation (Training time)
            if self.aug:
                trans_image = transform(image_pre_proc, image_size = image_pre_proc.shape)
                resized_image = tf.image.resize(trans_image, [image_size, image_size])[:, :, 0]
                resized_image = keras.applications.xception.preprocess_input(resized_image*255)
            else:
                resized_image = tf.image.resize(image_pre_proc, [image_size, image_size])[:, :, 0]
                resized_image = keras.applications.xception.preprocess_input(resized_image*255)
                
            out_img = tf.stack([resized_image, resized_image, resized_image], axis=-1)
            imgs_master.append(out_img)
        
#         imgs_master_cutup_mixup = cutmix_mixup(imgs_master)
        imgs_master_out = tf.stack(imgs_master)

        # target column
        FVC = self.dataset.loc[ids_tmp, 'FVC'].to_numpy(dtype=np.float32)

        # to tensor convert
        FVC = tf.convert_to_tensor(tf.constant(FVC))
        tab = tf.convert_to_tensor(tf.constant(tab))
        
        return [imgs_master_out, tab], FVC
 
    def on_epoch_end(self):
        self.indexes = np.arange(self.dataset.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
#         if self.shuffle:
#             np.random.shuffle(self.indexes)


## Model Architecture

In [110]:
# def get_dropout(input_tensor, p=0.5, mc=False):
#     if mc:
#         return keras.layers.Dropout(p)(input_tensor, training=True)
#     else:
#         return keras.layers.Dropout(p)(input_tensor)


# def get_model():

# with tpu_strategy.scope():
input_shape_image=(image_size,image_size,3)
input_image = keras.layers.Input(shape=input_shape_image, name="Image_Input")

########## Im,age Input ##########
# base_model = keras.applications.VGG19(weights="imagenet", include_top=False,
#             input_tensor=keras.layers.Input(shape=input_shape_image))
base_model = keras.applications.Xception(weights='imagenet', include_top=False, input_shape=input_shape_image)
base_model.trainable=False
base_model_output = base_model(input_image, training=False)
layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
out_dense_1 = keras.layers.Dense(64, name ="output_dense_img_1",activation="relu")(layer_global_pooling)  
bn_out_1 = keras.layers.BatchNormalization()(out_dense_1)


dropout_img_1 = keras.layers.Dropout(0.45, name="dropout_img_1")(bn_out_1)
# Monte Carlo Dropout

#     dropout_img_1 = get_dropout(bn_out_1, p=0.45, mc=mc)

out_dense_2 = keras.layers.Dense(16, name ="output_dense_img_2",activation="relu")(dropout_img_1)  
# out_image = keras.layers.Dense(16, name ="output_image",activation="relu")(dropout_img_2)  

########## Tabular Input ##########
tabular_inp= keras.layers.Input(shape=(8,))
# Concat - Image and Tabular
concat = keras.layers.Concatenate(name="concat")([out_dense_2, tabular_inp])
concat_out_1 = keras.layers.Dense(64, name="concat_out_2", activation="relu",
                                 kernel_regularizer=keras.regularizers.l2(0.1)
                                 )(concat)
bn_out_2 = keras.layers.BatchNormalization()(concat_out_1)
drop_out_img_tab_1 = keras.layers.Dropout(0.25, name="dropout_img_tab_2")(bn_out_2)
#     drop_out_img_tab_1 = get_dropout(bn_out_2, p=0.2, mc=mc)  
concat_out_2 = keras.layers.Dense(32, name="concat_out_3", activation="relu")(drop_out_img_tab_1)
p1 = keras.layers.Dense(3, activation="linear", name="p1")(concat_out_2)
p2 = keras.layers.Dense(3, activation="relu", name="p2")(concat_out_2)
preds = keras.layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                 name="preds")([p1, p2])

model = keras.Model(inputs = [input_image, tabular_inp], outputs = preds)
for layer in base_model.layers:
    layer.trainable = False


print(model.summary())
print(f" N Layers of BASE model --> {len(base_model.layers)}")
print(f" N Layers of our model --> {len(model.layers)}")
for index, layer in enumerate(model.layers):
    print(index, layer.name)

#     return model 

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image_Input (InputLayer)        [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
xception (Functional)           (None, 10, 10, 2048) 20861480    Image_Input[0][0]                
__________________________________________________________________________________________________
pretrained_global_avg_pooling ( (None, 2048)         0           xception[0][0]                   
__________________________________________________________________________________________________
output_dense_img_1 (Dense)      (None, 64)           131136      pretrained_global_avg_pooling[0][
_______________________________________________________________________________________

## RUN MODEL FOR INPUT FOLD

In [ ]:
# def run(model, fold):

# read folds data and get train & validation data 
# model = get_model()
df_folds = pd.read_csv(f"{OUTPUT_PATH}/osic_{n_folds}_folds.csv")
print(f"N Folds in Cross Validation --> {df_folds.fold.nunique()}")
df_train = df_folds[df_folds.fold !=fold].reset_index(drop=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
print("############ Before dropping Junk Patients ################")
print(f"Train Shape --> {df_train.shape}")
print(f"Number of unique patients in Train --> {df_train.Patient.nunique()}")
# get validation data
df_valid = df_folds[df_folds.fold==fold].reset_index(drop=True)  
print(f"Validation Shape --> {df_valid.shape}")
print(f"Number of unique patients in Validation --> {df_valid.Patient.nunique()}")


print("############ After dropping Junk Patients ################")


df_train = df_train[~df_train.Patient.isin(junk_patients)].reset_index(drop=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_valid = df_valid[~df_valid.Patient.isin(junk_patients)].reset_index(drop=True)

print(f"Train Shape --> {df_train.shape}")
print(f"Number of unique patients in Train --> {df_train.Patient.nunique()}")
print(f"Validation Shape --> {df_valid.shape}")
print(f"Number of unique patients in Validation --> {df_valid.Patient.nunique()}")


# compile model
# lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)
# optimizer = keras.optimizers.Adam(lr=0.03)
# optimizer = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# optimizer = keras.optimizers.SGD(lr=0.05, momentum=0.9)
# optimizer = keras.optimizers.Nadam(lr=0.003, beta_1=0.9, beta_2=0.999)


model.compile(loss=mloss(0.8), 
          optimizer=optimizer, metrics=[score])

# check point and early stopping

# fit model 
history = model.fit(DataGenerator(dataset=df_train, batch_size=bs, aug=True, shuffle=True),
                    validation_data=DataGenerator(dataset=df_valid, batch_size=bs, aug=False, shuffle=False), 
                    epochs=100, callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler])

# print validation data score
print(f"Validation Data Score --> {model.evaluate(DataGenerator(dataset=df_valid, batch_size=bs, aug=False, shuffle=False))}")

# return model via UDF
# return model

N Folds in Cross Validation --> 4
############ Before dropping Junk Patients ################
Train Shape --> (1113, 11)
Number of unique patients in Train --> 127
Validation Shape --> (369, 11)
Number of unique patients in Validation --> 42
############ After dropping Junk Patients ################
Train Shape --> (1096, 11)
Number of unique patients in Train --> 125
Validation Shape --> (369, 11)
Number of unique patients in Validation --> 42
Epoch 1/100
68/68 [==============================] - 52s 766ms/step - loss: 1086.0342 - score: 24.7801 - val_loss: 1058.7245 - val_score: 24.7981
Epoch 2/100
68/68 [==============================] - 52s 760ms/step - loss: 1055.3392 - score: 24.4163 - val_loss: 962.6561 - val_score: 18.4935
Epoch 3/100
68/68 [==============================] - 50s 742ms/step - loss: 934.8427 - score: 15.5604 - val_loss: 814.6988 - val_score: 11.7471
Epoch 4/100
68/68 [==============================] - 51s 755ms/step - loss: 691.2621 - score: 10.5226 - val_loss: 68

## *Validation Predictions*

In [ ]:
# df_valid_tmp = df_valid[['Patient', 'week', 'FVC', 'fold']]
# df_valid_tmp['confidence'] = valid_preds[:, 2] - valid_preds[:, 0]
# df_valid_tmp['pred_fvc'] = valid_preds[:, 1]
# df_valid_tmp['sigma_clipped'] = df_valid_tmp['confidence'].apply(lambda x: max(x, 70))
# df_valid_tmp['diff'] = abs(df_valid_tmp['FVC'] - df_valid_tmp['pred_fvc'])
# df_valid_tmp['delta'] = df_valid_tmp['diff'].apply(lambda x: min(x, 1000))
# df_valid_tmp['score'] = -math.sqrt(2)*df_valid_tmp['delta']/df_valid_tmp['sigma_clipped'] - np.log(math.sqrt(2)*df_valid_tmp['sigma_clipped'])
# df_valid_tmp.score.mean()
valid_preds = model.predict(DataGenerator(dataset=df_valid, batch_size=4, aug=True, shuffle=False))
print(valid_preds.shape)

# *Test Predictions*

In [ ]:
df_test = pd.read_csv(f"{OUTPUT_PATH}/osic_test.csv")
print(df_test.shape)
df_test.head()


## Test Results WITHOUT Augmentation on BASE FREEZED + CUSTOM MODEL

In [ ]:
test_pred = model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=False, shuffle=False))
df_test['confidence'] = test_pred[:, 2] - test_pred[:, 0]
df_test['pred_fvc'] = test_pred[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred.shape}")
print(f"\n**Overall Mean Score WITHOUT Augmentation on BASE FREEZED + CUSTOM MODEL --> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")
df_test.sample(frac=1).head(10)

## Test results WITH Augmentation on BASE FREEZED + CUSTOM MODEL

In [ ]:
test_pred_aug = np.stack([model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=True, shuffle=False)) for sample in range(TTA_STEPS)])
test_pred_mean = test_pred_aug.mean(axis=0)
df_test['confidence'] = test_pred_mean[:, 2] - test_pred_mean[:, 0]
df_test['pred_fvc'] = test_pred_mean[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred_aug.shape}")
print(f"\n**Overall Mean Score WITH Augmentation on BASE FREEZED + CUSTOM MODEL --> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")


## FINE TUNING BASE MODEL

In [ ]:
base_model.summary()

In [ ]:
base_model.trainable=True
print(f"N layers in base model --> {len(base_model.layers)}")
print(f"Freezing Top {trainable} layers")
for layer in base_model.layers[:(len(base_model.layers) - trainable)]:
        layer.trainable = False
for layer in base_model.layers[(len(base_model.layers) - trainable):]:
    layer.trainable = True


In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=early_stopping_fine_tuning, restore_best_weights=True)


optimizer = keras.optimizers.SGD(lr=(0.00125/10), momentum=0.9, nesterov=True)
model.compile(loss=mloss(0.8), optimizer=optimizer, metrics=[score])

In [ ]:
model.summary()

In [ ]:
history = model.fit(DataGenerator(dataset=df_train, batch_size=bs, aug=True, shuffle=True),
                    validation_data=DataGenerator(dataset=df_valid, batch_size=bs, aug=False, shuffle=False), 
                    epochs=100, callbacks=[checkpoint_cb, early_stopping_cb])

print(f"Validation Data Score --> {model.evaluate(DataGenerator(dataset=df_valid, batch_size=bs, aug=False, shuffle=False))}")


## Test Results WITHOUT Augmentation on FINE TUNED MODEL

In [ ]:
test_pred = model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=False, shuffle=False))
df_test['confidence'] = test_pred[:, 2] - test_pred[:, 0]
df_test['pred_fvc'] = test_pred[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred.shape}")
print(f"\n**Overall Mean Score WITHOUT Augmentation ON FINE TUNED MODEL--> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")
df_test.sample(frac=1).head(10)

## Test Results WITH Augmentation on FINE TUNED MODEL

In [ ]:
TTA_STEPS = 11

In [ ]:
test_pred_aug = np.stack([model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=True, shuffle=False)) for sample in range(TTA_STEPS)])
test_pred_mean = test_pred_aug.mean(axis=0)
df_test['confidence'] = test_pred_mean[:, 2] - test_pred_mean[:, 0]
df_test['pred_fvc'] = test_pred_mean[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred_aug.shape}")
print(f"\n**Overall Mean Score WITH Augmentation ON FINE TUNED MODEL--> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")


In [ ]:
print(np.mean(df_test.score))
df_test.groupby('Patient')['score'].mean()

In [ ]:
# -7.1704979101657065
# Patient
# ID00419637202311204720264   -7.184254
# ID00421637202311550012437   -7.128723
# ID00422637202311677017371   -6.614698
# ID00423637202312137826377   -7.785128
# ID00426637202313170790466   -7.077930
# Name: score, dtype: float64

In [106]:
# df_test['fold'] = fold
# df_test
# out_filename = f"/kaggle/working/pred_test_{n_folds}FOLDS_{EPOCHS}E_fold_{fold}.csv"
# print(out_filename)
# df_test.to_csv(out_filename, index=False)

In [67]:
# model.save_weights('tmp_model_fold_0.h5')

## PLAYGROUND

In [ ]:
# # model2 = tf.keras.models.load_model("/kaggle/working/osic_pat_level_image_index_4FOLDS_100E_f_0.h5", compile=False)
# # print('model summary after loading in a scope')
# # model2.summary()


# Patient
# ID00419637202311204720264   -8.151812
# ID00421637202311550012437   -7.574516
# ID00422637202311677017371   -7.408638
# ID00423637202312137826377   -7.992481
# ID00426637202313170790466   -7.528189
# Name: score, dtype: float64

In [ ]:
# a = model.predict(DataGenerator(dataset=df_sub, batch_size=10, aug=False, shuffle=False))
# print(a.shape)
# df_sub['confidence'] = a[:, 2] - a[:, 0]
# df_sub['pred_fvc'] = a[:, 1]
# df_sub = df_sub.drop('FVC', axis=1)
# out = pd.merge(df_sub, sub_mapping, on = ["Patient", "week"], how="inner")
# out[['Patient', 'Weeks', ]]
# df_sub = pd.read_csv(f"{OUTPUT_PATH}/osic_submission.csv")
# print(df_sub.shape)
# sub_mapping = pd.read_csv(f"{PATH_SUB}/osic_submission_mapping.csv")
# print(sub_mapping.shape)
# df_sub.head()
# preds_test_aug = np.stack([model2.predict(DataGenerator(dataset=df_test, batch_size=2, aug=True, shuffle=False)) for sample in range(25)])
# preds_test_mean = preds_test_aug.mean(axis=0)
# a = preds_test_mean
# a

In [ ]:
df_test

In [ ]:
# test_preds_mc_dropout = np.stack([model(DataGenerator(dataset=df_test, batch_size=4, aug=True, shuffle=False), training=True) for sample in range(10)])


In [ ]:
# # test_preds_mc_dropout.shape
# model(DataGenerator(dataset=df_test, batch_size=4, aug=True, shuffle=False), training=True)